In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense

In [30]:
class AlexNet():
    
    def __init__(self):
        self.model = Sequential()
        self.model.add(Conv2D(96, input_shape=(224,224,3), kernel_size=11, activation="relu"))
        self.model.add(MaxPool2D((3,3)))
        self.model.add(Conv2D(256, kernel_size=5, activation="relu"))
        self.model.add(MaxPool2D((3,3)))
        self.model.add(Conv2D(384, kernel_size=3, activation="relu"))
        self.model.add(MaxPool2D((3,3)))
        self.model.add(Conv2D(256, kernel_size=3, activation="relu"))
        self.model.add(MaxPool2D((3,3)))
        self.model.add(Dense(4096, activation="leaky_relu"))
        self.model.add(Dense(4096, activation="leaky_relu"))
        self.model.add(Dense(17, activation="linear"))
        self.model.compile(optimizer="sgd", loss="root_mean_squared_error", metrics="accuracy")

In [31]:
model = AlexNet()

In [32]:
model.model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_26 (Conv2D)          (None, 214, 214, 96)      34944     
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 71, 71, 96)       0         
 g2D)                                                            
                                                                 
 conv2d_27 (Conv2D)          (None, 67, 67, 256)       614656    
                                                                 
 max_pooling2d_22 (MaxPoolin  (None, 22, 22, 256)      0         
 g2D)                                                            
                                                                 
 conv2d_28 (Conv2D)          (None, 20, 20, 384)       885120    
                                                                 
 max_pooling2d_23 (MaxPoolin  (None, 6, 6, 384)      